# 0 Imports

In [1]:
# keras imports

import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import keras.backend as K
from keras.models import model_from_json

Using TensorFlow backend.


In [0]:
# tf.keras imports

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
import tensorflow.keras.backend as K
from tensorflow.keras.models import model_from_json

**NOTE about keras versus tf.keras**

tf.keras can run any keras-compatible code, but
1. version in latest TensorFlow may differ from latest Keras version in PyPi
2. when saving model weights, tf.keras defaults to checkpoint format. For HDF5, pass save_format='h5'

There might be some differences. "from keras.utils.vis_utils import model_to_dot" works but "from tensorflow.keras.utils.vis_utils import model_to_dot" doesn't.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
from IPython.display import SVG
import os
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
from matplotlib import pyplot as plt

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [0]:
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import cv2
import glob
import os
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split



from matplotlib.pyplot import imshow
%matplotlib inline

In [0]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

# 1 Loading data

In [0]:
def get_images(parent_directory):
    results = glob.glob(os.path.join(parent_directory,'*.jpg'))+glob.glob(os.path.join(parent_directory,'*.jpeg'))+glob.glob(os.path.join(parent_directory,'*.png'))
    images = []
    for result in results:
        images.append(cv2.imread(result))
    return images
  
def resize_crop(image,square_size):
    w = image.shape[1]
    h = image.shape[0]
    min_dim = min(w,h)  
    max_square_image = crop_center(image, min_dim, min_dim)
    result = cv2.resize(max_square_image,(square_size,square_size),0,0)
    return result
  
def crop_center(image,cropx,cropy):
    w = image.shape[1]
    h = image.shape[0]
    startx = w//2-(cropx//2)
    starty = h//2-(cropy//2)
    return image[starty:starty+cropy, startx:startx+cropx]

In [0]:
def get_classifier_data(image_folder, size = 256, verbose = True, channels="last",one_hot=True):
    folders = os.listdir(image_folder)
    if verbose:
        print("classes: {0}".format(folders))
    label_dict = {}

    for i, folder in enumerate(folders):
      files = glob.glob(os.path.join(image_folder, folder,'*.jpg'))+glob.glob(os.path.join(image_folder, folder,'*.jpeg'))+glob.glob(os.path.join(image_folder, folder,'*.png'))
      
      if channels == "last":
        folder_array=np.zeros((len(files),size,size,3))
        for j,file in enumerate(files):
          folder_array[j,:] = resize_crop(cv2.imread(file),size) 
          
      elif channels == "first":
        folder_array=np.zeros((len(files),3,size,size))
        for j,file in enumerate(files):
          folder_array[j,:] = np.transpose(resize_crop(cv2.imread(file),size),(2,0,1))      
      
      else:
        raise("channels must be 'first' or 'last'")
      
      folder_labels = np.array([i]*len(files))
      label_dict[str(i)] = folder
      
      if i == 0:
        input_array = folder_array
        input_labels = folder_labels
      else:
        input_array = np.concatenate([input_array,folder_array])
        input_labels = np.concatenate([input_labels, folder_labels])
        
      if one_hot:
        input_labels = convert_to_one_hot(input_labels,len(list(np.unique(input_labels)))
        
    return input_array, input_labels, label_dict
   
     

In [10]:
image_folder = "drive/My Drive/xylosai/tensorflow/images"
size = 256
channels = "last"
input_images, labels, label_dict = get_classifier_data(image_folder,channels=channels,size=size)

classes: ['car', 'plane', 'train']


In [0]:
labels = labels.T

In [30]:
print(input_images.shape)
print(labels.shape)
print(label_dict)

(515, 3, 256, 256)
(3, 515)
{'0': 'car', '1': 'plane', '2': 'train'}


# 2 Building Keras model and testing

## 2.1 Train/test split

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(input_images,labels)

In [13]:
print(input_images.shape)
print(labels.shape)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


(515, 3, 256, 256)
(515,)
(386, 3, 256, 256)
(129, 3, 256, 256)
(386,)
(129,)


## 2.2  model 1

In [0]:


def ModelCarPlaneTrain(input_shape = (256,256,3), classes = 3):
    
    X_input = Input(input_shape)
    
    X = Conv2D(160,(4,4),strides = (1,1),name='conv1',kernel_initializer = glorot_uniform(seed=0))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D((8, 8))(X)
    
    X = Conv2D(160,(2,2),strides = (1,1),name='conv2',kernel_initializer = glorot_uniform(seed=0))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((4, 4))(X)
    
    X = Conv2D(80,(2,2),strides = (1,1),name='conv3',kernel_initializer = glorot_uniform(seed=0))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((4, 4))(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc', kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name = 'ModelCarPlaneTrain-2layer')
    
    return model

In [0]:
model_1 = ModelCarPlaneTrain(input_shape=(size,size,3))

In [0]:
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model_1.count_params()

161923

In [21]:
X_train.shape

(386, 3, 256, 256)

In [0]:
X_train = np.transpose(X_train, (0,2,3,1))
X_test = np.transpose(X_test, (0,2,3,1))

In [39]:
model_1.fit(X_train,Y_train, epochs = 1, batch_size = 64)

Epoch 1/1
386/386 [==============================] - 90s 234ms/step - loss: 11.4831 - acc: 0.2876


In [0]:
preds_1 = model_1.evaluate(X_test,Y_test)
print ("Loss = " + str(preds_1[0]))
print ("Test Accuracy = " + str(preds_1[1]))

130/130 [==============================] - 1s 10ms/step
Loss = 1.0909130499913142
Test Accuracy = 0.3230769230769231


## 2.3 model 2

In [0]:
# This model is less complex and has same accuracy

def ModelCarPlaneTrain_2(input_shape = (256,256,3), classes = 3):
    
    X_input = Input(input_shape)
    
    X = Conv2D(8,(4,4),strides = (1,1),name='conv1',kernel_initializer = glorot_uniform(seed=0))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D((16, 16))(X)
    
    #X = Conv2D(32,(2,2),strides = (1,1),name='conv2',kernel_initializer = glorot_uniform(seed=0))(X)
    #X = Activation('relu')(X)
    #X = MaxPooling2D((4, 4))(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc', kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name = 'ModelCarPlaneTrain-2layer')
    
    return model

In [0]:
model_2 = ModelCarPlaneTrain_2(input_shape=(size,size,3))
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#without augmentation
model_2.fit(X_train,Y_train, epochs = 1, batch_size = 16)

#note: when switching, do not fit model twice (ensure correct comparison) (won't happen when re-running this cell)

#with augmentation (vertical mirroring)
#model_2.fit(X_train_augm,Y_train_augm, epochs = 60, batch_size = 16)

Epoch 1/1
388/388 [==============================] - 9s 22ms/step - loss: 0.9500 - acc: 0.5284


In [0]:
preds_2 = model_2.evaluate(X_test,Y_test)
print ("Loss = " + str(preds_2[0]))
print ("Test Accuracy = " + str(preds_2[1]))

#augmentation does not seem to improve accuracy significantly

130/130 [==============================] - 1s 10ms/step
Loss = 0.8378040020282452
Test Accuracy = 0.6307692307692307


## 2.4 exporting and loading of keras model

**2.4.1 Method 1 (model and weights in seperate files) **

We are passing save_format='h5' explicitly, because when using tf.keras it would default to TensorFlow's checkpoint format.

In [0]:
#exporting the second model to json

model_2_json = model_2.to_json()
with open("keras_model_2.json","w") as file:
    file.write(model_2_json)   
model_2.save_weights("keras_model_2.h5",save_format='h5')


In [0]:
#loading and using model

file = open('keras_model_2.json', 'r')
loaded_model_2_json = file.read()
file.close()
loaded_model_2 = model_from_json(loaded_model_2_json)
loaded_model_2.load_weights("keras_model_2.h5")

loaded_model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
preds = loaded_model_2.evaluate(X_test,Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

130/130 [==============================] - 1s 10ms/step
Loss = 0.8378040020282452
Test Accuracy = 0.6307692307692307


**2.4.1 Method 2 (model and weights in one file, easier)**

In [0]:
keras.models.save_model(model_2,"keras_model_2_full.h5")

In [0]:
loaded_model_2 = load_model("keras_model_2_full.h5")

In [0]:
preds = loaded_model_2.evaluate(X_test,Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

130/130 [==============================] - 1s 10ms/step
Loss = 0.8378040020282452
Test Accuracy = 0.6307692307692307


# 3 Autokeras

## 3.1 Just structuring the data in the Autokeras way... REMOVE THIS FROM TRAINING VERSION

Images must be in one folder, and a csv-file is required with header "File Name, Label" that maps file names to label strings. Images must all have equal pixel size, otherwise load_image_dataset() will not raise error but will return a wrong result (a numpy array of numpy arrays) so that fitting fails.

First, copying all seperate image folders and renaming these folders to to "..._renamed"

import glob
import os

filenames_car = glob.glob('fotos_auto_renamed/*.jpg')+glob.glob('fotos_auto_renamed/*.jpeg')+glob.glob('fotos_auto_renamed/*.png')
filenames_plane = glob.glob('fotos_vliegtuig_renamed/*.jpg')+glob.glob('fotos_vliegtuig_renamed/*.jpeg')+glob.glob('fotos_vliegtuig_renamed/*.png')
filenames_train = glob.glob('fotos_trein_renamed/*.jpg')+glob.glob('fotos_trein_renamed/*.jpeg')+glob.glob('fotos_trein_renamed/*.png')

with open("labels.csv", 'a') as label_file:
    label_file.write("File Name,Label\n")
    for filename in filenames_car:
        os.rename(filename, filename.split('\\')[0]+'\\car_'+filename.split('\\')[1])
        label_file.write("{0},{1}\n".format("car_"+filename.split('\\')[1],'car'))
    for filename in filenames_plane:
        os.rename(filename, filename.split('\\')[0]+'\\plane_'+filename.split('\\')[1])
        label_file.write("{0},{1}\n".format("plane_"+filename.split('\\')[1],'plane'))
    for filename in filenames_train:
        os.rename(filename, filename.split('\\')[0]+'\\train_'+filename.split('\\')[1])
        label_file.write("{0},{1}\n".format("train_"+filename.split('\\')[1],'train'))

Now add all files to one folder images_all/

for filename in glob.glob('images_all/*'):
    raw_image = cv2.imread(filename)
    resized_image = resize_crop(raw_image,256)
    cv2.imwrite(filename,resized_image)

test1 = cv2.imread(glob.glob('images_all/*')[1])
test2 = cv2.imread(glob.glob('images_all/*')[2])
print(test1.shape)
print(test2.shape)

## 3.2 Building and training Autokeras model

I have run code below for several hours. Did not stop after time limit and also produced disappointing results.

In [0]:
import autokeras as ak
from autokeras.classifier import ImageClassifier,load_image_dataset
from sklearn.model_selection import train_test_split

In [0]:
X, Y = load_image_dataset(csv_file_path="labels.csv",images_path="images_all")

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [0]:
classifier = ImageClassifier(verbose=True)

In [0]:
classifier.fit(X_train,Y_train,time_limit = 2 * 60 * 60)

Initializing search.
Initialization finished.
Training model  0
Saving model.
Model ID: 0
Loss: 0.7097633161495641
Accuracy 0.6082474226804123
Training model  1
Father ID:  0
[('to_add_skip_model', 1, 6), ('to_wider_model', 6, 64), ('to_concat_skip_model', 1, 6)]


In [0]:
classifier.final_fit(X_train,Y_train, X_test, Y_test, retrain=True)

In [0]:
y = classifier.evaluate(X_test, Y_test)

In [0]:
print(time.time())